In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("who\FullTable_WHO.csv")
df = df.drop(["Continent", "Income Per Person"], axis=1) # not relevant for LR
df.dropna(axis=0, thresh=25, inplace = True) # dropping countries with too many missing consumption values
df

,Country,Beverage Types,2019,2018,2017,2016,2015,2014,2013,2012,...,1970,1969,1968,1967,1966,1965,1964,1963,1962,1961
0,Afghanistan,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Afghanistan,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,Afghanistan,3,0.01,0.01,0.01,0.01,0.00,0.01,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Afghanistan,4,0.01,0.01,0.01,0.01,0.00,0.01,0.01,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Albania,3,1.43,1.50,1.56,1.61,1.68,1.74,1.79,1.88,...,0.79,0.73,0.55,0.49,0.38,0.33,0.26,0.20,0.41,0.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,Zambia,3,0.36,0.25,0.22,0.21,0.21,0.18,0.17,0.17,...,0.04,0.04,0.05,0.07,0.09,0.09,7.59,0.09,0.09,0.09
748,Zimbabwe,4,3.11,3.10,3.82,3.76,3.84,4.22,4.11,3.93,...,5.99,5.08,6.04,5.49,6.05,4.87,4.56,4.63,5.05,4.24
749,Zimbabwe,1,1.20,1.20,1.93,1.93,1.93,2.23,2.12,2.07,...,0.57,0.55,0.54,0.54,0.54,0.22,0.21,0.25,0.24,0.25
750,Zimbabwe,2,0.05,0.04,0.05,0.05,0.06,0.06,0.05,0.05,...,0.00,0.00,0.00,0.00,0.01,0.04,0.04,0.05,0.02,0.04


In [3]:
df_new = pd.DataFrame()
for c in df.Country:
    df_c = df.loc[df['Country'] == c]
    df_c = df_c.sort_values(by = 'Beverage Types') #sorting every country to the same beverage order
    #replacing every NaN value with the year's median value for each country and each beverage
    m = df_c.iloc[:, 2:].median(axis=1)
    for i, col in enumerate(df_c):
        # using i allows for duplicate columns
        # inplace *may* not always work here, so IMO the next line is preferred
        # df.iloc[:, i].fillna(m, inplace=True)
        df_c.iloc[:, i] = df_c.iloc[:, i].fillna(m)
    
    for col in df_c.loc[:, (df.columns!='Country') & (df.columns!='Beverage Types')]: #ignoring first 2 columns
        df_row = df_c.loc[:,df_c.columns==col].T
        df_new = df_new.append(pd.Series([c,col, df_row.values.tolist()[0]], index=['Country','Year','values']),
                               ignore_index=True)
        
# creating a new DF from all subdataframes (for every country) 
df_new2 = pd.DataFrame(df_new['values'].to_list(), columns=['Beer consumption','Wine consumption','Spirit consumption','Total consumption'])
df_new = pd.concat([df_new, df_new2], axis=1)
df_new = df_new.drop(columns=['values'])
df_new = df_new.drop_duplicates()

In [4]:
df_new = df_new.reset_index(drop=True)

In [5]:
df_new.to_csv("data_for_lr.csv")

In [6]:
#united states data:
df = pd.read_csv("niaaa.nih\\niaaa_nih.csv")
df = df.drop(["Unnamed: 0"], axis=1)
df.dropna(axis=0, thresh=25, inplace = True)
df

,Country,Drink type,2019,2018,2017,2016,2015,2014,2013,2012,...,1986,1985,1984,1983,1982,1981,1980,1979,1978,1977
0,Alabama,1,1.02,1.04,1.07,1.09,1.11,1.13,1.14,1.16,...,1.10,1.07,1.03,1.01,1.00,1.00,0.95,0.96,0.95,0.97
1,Alabama,2,0.26,0.26,0.26,0.26,0.26,0.25,0.25,0.24,...,0.18,0.17,0.19,0.17,0.18,0.19,0.16,0.12,0.12,0.12
2,Alabama,3,0.71,0.69,0.67,0.65,0.64,0.62,0.60,0.60,...,0.65,0.67,0.69,0.72,0.72,0.73,0.74,0.82,0.85,0.82
3,Alabama,4,2.00,1.99,2.00,2.01,2.01,2.00,1.99,2.00,...,1.92,1.92,1.91,1.90,1.90,1.91,1.86,1.90,1.92,1.92
4,Alaska,1,1.08,1.07,1.06,1.15,1.11,1.11,1.07,1.17,...,1.65,1.63,1.67,1.71,1.69,1.68,1.54,1.48,1.39,1.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Wisconsin,4,2.95,2.94,2.99,2.99,2.98,2.98,3.00,3.02,...,3.18,3.20,3.21,3.35,3.30,3.48,3.45,3.39,3.31,3.37
200,Wyoming,1,1.23,1.22,1.17,1.18,1.25,1.26,1.25,1.32,...,1.51,1.59,1.61,1.68,1.78,1.91,1.85,1.86,1.81,1.78
201,Wyoming,2,0.31,0.32,0.32,0.32,0.31,0.31,0.29,0.28,...,0.27,0.25,0.23,0.22,0.24,0.24,0.24,0.22,0.22,0.20
202,Wyoming,3,1.29,1.24,1.19,1.17,1.18,1.16,1.13,1.13,...,0.96,1.05,1.06,1.11,1.20,1.27,1.32,1.30,1.35,1.31


In [7]:
df_new = pd.DataFrame()
for c in df.Country:
    df_c = df.loc[df['Country'] == c]
    df_c = df_c.sort_values(by = 'Drink type') #sorting every country to the same beverage order
    #replacing every NaN value with the year's median value for each country and each beverage
    m = df_c.iloc[:, 2:].median(axis=1)
    for i, col in enumerate(df_c):
        # using i allows for duplicate columns
        # inplace *may* not always work here, so IMO the next line is preferred
        # df.iloc[:, i].fillna(m, inplace=True)
        df_c.iloc[:, i] = df_c.iloc[:, i].fillna(m)
    
    for col in df_c.loc[:, (df.columns!='Country') & (df.columns!='Drink type')]: #ignoring first 2 columns
        df_row = df_c.loc[:,df_c.columns==col].T
        df_new = df_new.append(pd.Series([c,col, df_row.values.tolist()[0]], index=['Country','Year','values']),
                               ignore_index=True)
        
# creating a new DF from all subdataframes (for every country) 
df_new2 = pd.DataFrame(df_new['values'].to_list(), columns=['Beer consumption','Wine consumption','Spirit consumption','Total consumption'])
df_new = pd.concat([df_new, df_new2], axis=1)
df_new = df_new.drop(columns=['values'])
df_new = df_new.drop_duplicates()

In [8]:
df_new = df_new.reset_index(drop=True)

In [9]:
df_new.to_csv("united_states_data_for_lr.csv")

In [10]:
df2 = pd.read_csv("united_states_data_for_lr.csv")
df2 = df2.drop(["Unnamed: 0"], axis=1)
df2

,Country,Year,Beer consumption,Wine consumption,Spirit consumption,Total consumption
0,Alabama,2019,1.02,0.26,0.71,2.00
1,Alabama,2018,1.04,0.26,0.69,1.99
2,Alabama,2017,1.07,0.26,0.67,2.00
3,Alabama,2016,1.09,0.26,0.65,2.01
4,Alabama,2015,1.11,0.26,0.64,2.01
...,...,...,...,...,...,...
2188,Wyoming,1981,1.91,0.24,1.27,3.43
2189,Wyoming,1980,1.85,0.24,1.32,3.41
2190,Wyoming,1979,1.86,0.22,1.30,3.39
2191,Wyoming,1978,1.81,0.22,1.35,3.38
